# Challenge Machine Learning 
# Florian André 3B IA

Dataset utilisé : Backpack Prediction Challenge trouvé sur Kaggle 
URL : https://www.kaggle.com/competitions/playground-series-s5e2/overview


## Explication du dataset
Ce dataset est composé de 4 fichiers 
- train.csv : notre dataset d'entraînement
- train_extra.csv : dataset d'entraînement supplémentaire (avec erreurs)
- test.csv : dataset pour tester notre modèle
- sample_submission : dataset contenant l'id des sacs testés ainsi que leur prédiction de prix

In [121]:
## Liste des bibliothèques utilisées
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras import backend as K
import seaborn as sns
import keras
import numpy as np

## Importation et nettoyage des données

Nous allons dans un premier temps importer les données d'entraînement de notre fichier train.csv

In [122]:
path = "/home/florian-andr/Downloads/playground-series-s5e2"
df_train = pd.read_csv(f"{path}/train.csv")
print(df_train.head())
print(df_train.shape)

   id         Brand Material    Size  Compartments Laptop Compartment  \
0   0      Jansport  Leather  Medium           7.0                Yes   
1   1      Jansport   Canvas   Small          10.0                Yes   
2   2  Under Armour  Leather   Small           2.0                Yes   
3   3          Nike    Nylon   Small           8.0                Yes   
4   4        Adidas   Canvas  Medium           1.0                Yes   

  Waterproof      Style  Color  Weight Capacity (kg)      Price  
0         No       Tote  Black             11.611723  112.15875  
1        Yes  Messenger  Green             27.078537   68.88056  
2         No  Messenger    Red             16.643760   39.17320  
3         No  Messenger  Green             12.937220   80.60793  
4        Yes  Messenger  Green             17.749338   86.02312  
(300000, 11)


Procédons maintenant au nettoyage des données : en effet, certaines lignes comportent des valeurs nulles qu'il faut éliminer avant d'effectuer l'entraînement

In [123]:
df_train.dropna(inplace=True)
print(df_train.head())
print(df_train.shape)

   id         Brand Material    Size  Compartments Laptop Compartment  \
0   0      Jansport  Leather  Medium           7.0                Yes   
1   1      Jansport   Canvas   Small          10.0                Yes   
2   2  Under Armour  Leather   Small           2.0                Yes   
3   3          Nike    Nylon   Small           8.0                Yes   
4   4        Adidas   Canvas  Medium           1.0                Yes   

  Waterproof      Style  Color  Weight Capacity (kg)      Price  
0         No       Tote  Black             11.611723  112.15875  
1        Yes  Messenger  Green             27.078537   68.88056  
2         No  Messenger    Red             16.643760   39.17320  
3         No  Messenger  Green             12.937220   80.60793  
4        Yes  Messenger  Green             17.749338   86.02312  
(246686, 11)


Il nous faut aussi éliminer les valeurs aberrantes sur "Weight Capacity (kg)" et "Price". On utilisera le z-score pour procéder à l'élimination de ces valeurs. 
Nous avons divisé le dataset en 2 quartiles (25% et 75%) sur la capacité de poids et le prix. Chaque valeur qui dépassera de 50% l'écart inter-quartile sera écartée
du dataset d'entraînement final.

In [124]:
QW1 = df_train["Weight Capacity (kg)"].quantile(0.25)
QW3 = df_train["Weight Capacity (kg)"].quantile(0.75)
IQWR = QW3 - QW1

QP1 = df_train["Price"].quantile(0.25)
QP3 = df_train["Price"].quantile(0.75)
IQPR = QP3 - QP1

threshold = 0.5
df_train = df_train[(df_train["Weight Capacity (kg)"] > (QW1 - threshold * IQWR)) & (df_train["Weight Capacity (kg)"] < (QW3 + threshold * IQWR))]
df_train = df_train[(df_train["Price"] > (QP1 - threshold * IQPR)) & (df_train["Price"] < (QP3 + threshold * IQPR))]
print(df_train.shape)

(234381, 11)


## Prétraitement des données (Data pre-processing)
Maintenant que notre dataset d'entraînement a été nettoyé, il faut prétraiter nos données via le feature engineering. On va utiliser une Random Forest ainsi que la méthode Feature Importance afin de ne garder que les variables les plus pertinentes pour notre modèle

On doit d'abord encoder les variables catégorielles de notre dataset

In [125]:

df_train["Brand_numeric"] = pd.factorize(df_train["Brand"])[0]
df_train["Material_numeric"] = pd.factorize(df_train["Material"])[0]
df_train["Size_numeric"] = pd.factorize(df_train["Size"])[0]
df_train["Laptop Compartment_numeric"] = pd.factorize(df_train["Laptop Compartment"])[0]
df_train["Waterproof_numeric"] = pd.factorize(df_train["Waterproof"])[0]
df_train["Style_numeric"] = pd.factorize(df_train["Style"])[0]
df_train["Color_numeric"] = pd.factorize(df_train["Color"])[0]

df_train.drop(columns=["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"], inplace=True)
print(df_train.head())


   id  Compartments  Weight Capacity (kg)      Price  Brand_numeric  \
0   0           7.0             11.611723  112.15875              0   
1   1          10.0             27.078537   68.88056              0   
2   2           2.0             16.643760   39.17320              1   
3   3           8.0             12.937220   80.60793              2   
4   4           1.0             17.749338   86.02312              3   

   Material_numeric  Size_numeric  Laptop Compartment_numeric  \
0                 0             0                           0   
1                 1             1                           0   
2                 0             1                           0   
3                 2             1                           0   
4                 1             0                           0   

   Waterproof_numeric  Style_numeric  Color_numeric  
0                   0              0              0  
1                   1              1              1  
2                   

Une fois les variables encodées, il est possible d'utiliser RandomForest pour avoir la Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
X_train = df_train.drop(columns=["id","Price"])
y_train = df_train["Price"]


rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

feature_importances = rf.feature_importances_


In [127]:
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

In [128]:
importance_df

,Feature,Importance
0,Compartments,0.127694
1,Weight Capacity (kg),0.482544
2,Brand_numeric,0.067224
3,Material_numeric,0.042666
4,Size_numeric,0.061745
5,Laptop Compartment_numeric,0.039072
6,Waterproof_numeric,0.036080
7,Style_numeric,0.061707
8,Color_numeric,0.081268


In [129]:
top_n_features = 6
top_features = importance_df.head(top_n_features)['Feature'].values

In [136]:
X_train = df_train[top_features]

## Entraînement du modèle

une fois les features les plus importantes sélectionnées, il est possible d'entraîner notre modèle à l'aide d'un perceptron multi-couches (MLP)

In [139]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

rgr = MLPRegressor(random_state=42, solver='adam')
rgr.fit(X_train, y_train)


MLPRegressor(random_state=42)

## Evaluation du modèle
Une fois le modèle entraîné, il ne nous reste plus qu'à l'évaluer sur le dataset de test

In [ ]:
df_test = pd.read_csv(f"{path}/test.csv")
X_test = df_test.drop(["id"], axis=1)
